# wps_update_metadata

#### wps_update_metadata is a process that runs the [update_metadata](https://github.com/pacificclimate/climate-explorer-data-prep#update_metadata-update-metadata-in-a-netcdf-file) module of PCIC Climate Explorer Data Preparation Tools. Here, the client will try to connect to a remote Thunderbird instance using the url parameter.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from birdy import WPSClient
from pkg_resources import resource_filename
import os
from wps_tools.utils import copy_http_content
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "thunderbird":
    os.chdir('../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thunderbird/wps'
thunderbird = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [ ]:
# NBVAL_IGNORE_OUTPUT
thunderbird.update_metadata?

#### We can use the docstring to ensure we provide the appropriate parameters.

In [ ]:
# run update_metadata with yaml text and opendap netcdf inputs
updates_text = '''
global:
    - institute_id:
    - institute_ID: PCIC
    - address: University House 1 
    - Institution: <- institution
'''
opendap_netcdf = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc"
opendap_output = thunderbird.update_metadata(updates = updates_text, netcdf = opendap_netcdf)

In [ ]:
# run update_metadata with yaml file and local netcdf inputs
updates_file = resource_filename('tests', 'metadata-conversion/simple-conversion.yaml')
local_netcdf = resource_filename('tests', 'data/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc')
local_output = thunderbird.update_metadata(updates = updates_file, netcdf = local_netcdf)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [ ]:
input_data = Dataset("https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc")

# Expected output metadata determined from the input yaml file
# global:
#     - institute_id:
#     - institute_ID: PCIC
#     - address: University House 1 
#     - Institution: <- institution
expected = {
    "institute_ID": "PCIC",
    "address": "University House 1",
    "Institution": input_data.institution,
}

In [ ]:
def test_metadata(output_data):
    with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
        output_data = Dataset(copy_http_content(output.get()[0], tmp_file))

    # updated metadata
    metadata = {
        "institute_ID": output_data.institute_ID,
        "address": output_data.address,
        "Institution": output_data.Institution,
    }

    assert metadata == expected

In [ ]:
test_metadata(opendap_output)
test_metadata(local_output)